Проверка подсчета свертки сечения с потоком с помощью разных методик интегрирования

In [1]:
import math
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
%pylab inline

from scipy.integrate import quad, fixed_quad, romberg
import scipy.integrate as sc_in

Populating the interactive namespace from numpy and matplotlib


In [2]:
Q_EC_76As=0.9233 #in MeV
Z_76As=33
log_ft_76As=5.0

Q_EC_37Ar=0.81387 #in MeV
Z_37Ar=18
log_ft_37Ar=5.1

g_a_g_v=1.297 #ratio of vector and axial coupling constants

In [3]:
def Q_k_func(E_k, Q_EC):
    #Считаем порог реакции, E_k - энергия возбужденного состояния
    #E_k in KeV
    return Q_EC+E_k/1000

Привиденная энергия вылетающего электрона: $\epsilon_e = \frac{E_{\nu}-Q_k}{m_e} + 1$

In [4]:
def e_e_func(E_nu, E_k, Q_EC):
    #Считаем привиденную энергию вылетающего электрона. Если энергия налетающего нейтрино E_nu
    #меньше порога, возвращаем 0 для обнуления итогового сечения
    #E_nu in MeV
    q=Q_k_func(E_k, Q_EC) #порог реакции
    if E_nu>q: return 1+(E_nu-q)/0.511
    else: return 0 

Привиденный импульс электрона: $\pi_e=\sqrt{\epsilon_e^2 -1}$

In [5]:
def Pi_e_func(e_e):
    #Привиденный импульс электрона, вычисляется через привиденную энергию; ** означает степень  
    return np.sqrt(np.square(e_e) - 1)

In [6]:
np.square(3)

9

### Fermi-function from Numerical Tables for Beta-Decay and Electron Capture

In [7]:
Fermi_func_new_data = pd.read_excel('../../fermi_func_tables/from_1968/13.xlsx')
Fermi_func_new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 14 columns):
P              49 non-null float64
F0L0           49 non-null float64
L0             48 non-null object
1z             48 non-null float64
13             48 non-null float64
ELECTRON
14    48 non-null float64
Z= 33
Izt      48 non-null float64
A= 77          48 non-null float64
A,             48 non-null float64
A2             48 non-null float64
VI2            48 non-null float64
912            48 non-null float64
12             48 non-null float64
Unnamed: 13    48 non-null float64
dtypes: float64(13), object(1)
memory usage: 5.4+ KB


In [8]:
#e_e = np.linspace(0,10,100, endpoint=False)
#Z - дочернего ядра, т.е 76As

e_e3 = np.sqrt( np.square( Fermi_func_new_data['P'].values ) + 1 )
def num_tables_func(x):
    #x in units of mc^2
    return np.interp(x=x, xp=e_e3, fp=Fermi_func_new_data['F0L0'].values )

#Num_tab =  num_tables_func( e_e )

### B(GT)

In [9]:
BGT=pd.read_csv('../B(GT) peaks processing/clear_B(GT).csv') #таблица со значениями B(GT)
BGT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 5 columns):
Unnamed: 0        73 non-null int64
Ex                73 non-null float64
Jpi               73 non-null object
diff cross-sec    73 non-null float64
B(GT)             73 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 2.9+ KB


In [10]:
BGT['B(GT)'] = BGT['B(GT)']/10

### BS05_OP flux

In [11]:
BS05_OP=pd.read_csv('../Vyborov_results/Fluxes/AllFluxes_BS05(OP).csv', sep=';')
BS05_OP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
energy    2000 non-null float64
pp        2000 non-null float64
hep       2000 non-null float64
N         2000 non-null float64
F         2000 non-null float64
O         2000 non-null float64
B         2000 non-null float64
dtypes: float64(7)
memory usage: 109.5 KB


### Cross-section and total rate

Сечение по Иджири $\sigma_k={(1.597\times10^{-44}\ cm^2)}\epsilon_e \pi_e m_e^2 F(Z,E_e) [B(F)_k + (\frac{g_a}{g_v})^2 B(GT)_k] $ 

In [12]:
#math.pow(0.511,2)*math.pow(g_a_g_v,2)*1.597*math.pow(10,-44)

def sigma_k(E_nu, E_k, BGT_k, Z, Q_EC):
    #сечение k-го возб. состояния по Иджири
    Q = Q_k_func(E_k, Q_EC) #порог
    if E_nu > Q:
        e_e = (E_nu-Q)/0.511 + 1
        Pi_e=Pi_e_func(e_e)
        F_col=num_tables_func(e_e)#ферми функция
        return e_e*Pi_e*F_col*BGT_k
    else: return 0

In [13]:
def flux(Energy, flux_name):
    return np.interp(x=Energy, xp=BS05_OP['energy'], fp=BS05_OP[flux_name] )

In [14]:
def integrand(E_nu, E_k, BGT_k, flux_name):
    return sigma_k(E_nu, E_k, BGT_k, Z_76As, Q_EC_76As)*flux(E_nu, flux_name)

In [15]:
const = math.pow(0.511,2)*math.pow(g_a_g_v,2)*1.597*math.pow(10,-44)

In [16]:
def R_k_ab(E_k, BGT_k, flux_name, a, b):
#    return romberg(integrand, 0, 20, args=(E_k, BGT_k, flux_name), divmax=10)
    return quad(integrand, 0, 20, args=(E_k, BGT_k, flux_name))
#    result = quad(integrand, 0, 20, args=(E_k, BGT_k, flux_name), full_output=True)
#    return result
#    return romberg(integrand, a, b, args=(E_k, BGT_k, flux_name))

In [17]:
#R_k(BGT['Ex'].values[0], BGT['B(GT)'].values[0], 'B')

In [18]:
def R_k(E_k, BGT_k, flux_name):
    return sum([R_k_ab(E_k, BGT_k, flux_name, a, b) for (a,b) 
                in zip(np.arange(0,20,1), np.arange(1,21,1))])

In [19]:
def R(flux_name):
    return sum([R_k(Ex, B_GT, flux_name) for (Ex, B_GT) in 
                zip(BGT['Ex'].values, BGT['B(GT)'].values)]) * const * math.pow(10,36)

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
for flux_name in list(BS05_OP.columns)[1:]:
    print( flux_name, R(flux_name) )

pp 0.0
hep 0.991303461369
N 0.0
F 0.397431118789
O 15.6354139479
B 298.668293272


romberg result

pp 0.0  
hep 0.0494961956331  
N 0.0  
F 0.0219029058787  
O 0.860019416531  
B 14.9325529493  

quad result

pp 0.0  
hep 0.0495011694405  
N 0.102464290385  
F 0.0219173456067  
O 0.860628316532  
B 14.9325574868  

In [22]:
BS05_OP_convolution = {'B':14.9325450351,
                       'pp':0.0,
                       'O':0.860093202034,
                       'F':0.0219055356605,
                       'N':0.101981181764,
                       'hep':0.0494961877882,
                       'pep':1.43946621379
                      }